In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('data.csv', encoding='latin-1')

# Displaying the first few rows of the DataFrame
print(df.head())


In [ ]:
df.columns

In [ ]:
df.describe

# **EDA**

In [ ]:
# Basic overview
print(df.info())

In [ ]:
# checking if there are any Missing values
print(df.isnull().sum())

In [ ]:
#checkinging missing values description(Products)
# Displaying rows with missing descriptions
missing_desc = df[df['Description'].isnull()]
print(missing_desc)


We are having 1454 null values in  Description column so we have to deal with it before we proceed further

In [ ]:
# Replace missing descriptions with "not mentioned" and inpulus will make this change in our original DataFrame
df['Description'].fillna('not mentioned', inplace=True)

# Verify the changes
print(df.info())


In [ ]:
# for checking rows with missing CustomerID so we can work on that, like how to manage these values
missing_customer_id = df[df['CustomerID'].isnull()]
print(missing_customer_id)


In [ ]:
#There are few non nomeric values in our invoce column
# Convert 'InvoiceNo' to numeric, setting non-numeric values to NaN
numeric_invoice = pd.to_numeric(df['InvoiceNo'], errors='coerce')
# Count the total non-numeric values
non_numeric_count = numeric_invoice.isna().sum()
# Displaying the result
print(f'Total non-numeric values in InvoiceNo: {non_numeric_count}')


In [ ]:
# Convert 'InvoiceNo' to numeric, setting non-numeric values to (-1) so we can perform numercal procedures
# we have used (errors='coerce') to avoid from error incase there is some non numeric value in our data whic can cause Nan (not a number) error
df['InvoiceNo'] = pd.to_numeric(df['InvoiceNo'], errors='coerce').fillna(-1).astype(int)

# Verify the changes
print(df.info())


Now we have no more null values in our data

In [ ]:
# Replace missing CustomerID with InvoiceNo + 1 so we can fill the gap
df['CustomerID'].fillna(df['InvoiceNo'] + 1, inplace=True)

# Verify the changes
print(df.info())


In [ ]:
# Convert 'InvoiceDate' to datetime
# we have used (errors='coerce') to avoid from error incase there is some non numeric value in our data whic can cause Nan (not a number) error
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

# Verify the changes
print(df.info())


In [ ]:
df.dtypes

In [ ]:
# Convert 'InvoiceDate' to datetime and extract the date
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate']).dt.date

# Verify the changes
print(df.info())


we still have invoice date column in object data type

In [ ]:
df.dtypes

In [ ]:
# Convert 'InvoiceDate' to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Verify the changes
print(df.dtypes)


In [ ]:
# Print the 'InvoiceDate' column
print(df['InvoiceDate'])



# **Recency**

In [ ]:
# Find the maximum date in the entire dataset
max_date_all = df['InvoiceDate'].max()
print(max_date_all)

In [ ]:
# ReAssigning column names in the original DataFrame
df.columns = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

# Creating a new DataFrame with updated column names
#making new data frame with Maxdate heading
max_date_df = df.groupby('CustomerID')['InvoiceDate'].max().reset_index(name='MaxDate')


In [ ]:
# Calculating Recency based on the maximum date for each customer
max_date_df = df.groupby('CustomerID')['InvoiceDate'].max().reset_index()
max_date_df.columns = ['CustomerID', 'MaxDate']
print(max_date_df)

In [ ]:
# Merge max_date_df into the original DataFrame df
df = pd.merge(df, max_date_df, on='CustomerID', how='left')

# Calculating Recency using the maximum date from the entire dataset
df['Recency'] = (max_date_all - df['MaxDate']).dt.days


In [ ]:
df.columns

In [ ]:
# Calculating Recency using the maximum date from the entire dataset
df['Recency'] = (max_date_all - df['MaxDate']).dt.days


In [ ]:
# Drop unnecessary MaxDate column and saving the data frame
df.drop(['MaxDate'], axis=1, inplace=True)

# Displaying the updated DataFrame
print("\nRecency Calculation using Maximum Date in the Entire Dataset")
print(df[['CustomerID', 'Recency']].head())

In [ ]:
df.columns

# **Frequency**

In [ ]:
# Calculating the Frequency of orders
frequency_df = df.groupby('CustomerID')['InvoiceNo'].nunique().reset_index()
frequency_df.columns = ['CustomerID', 'Frequency']
print("Step 1: Frequency Calculation")
print(frequency_df.head())


In [ ]:
# Merging the Frequency values into the main DataFrame
df = pd.merge(df, frequency_df, on='CustomerID', how='left')

In [ ]:
df.columns

In [ ]:
# Displaying the updated DataFrame
print("\nFrequency Calculation")
print(df[['CustomerID', 'Recency', 'Frequency']].head())

In [ ]:
df.dtypes

***Monetary***

In [ ]:
# Calculating the Monetary Value
df['Monetary'] = df['Quantity'] * df['UnitPrice']
print("\nStep 2: Monetary Calculation")
print(df[['CustomerID', 'Monetary']].head())


In [ ]:
# Making new Column Monetary
df['Monetary'] = df['Quantity'] * df['UnitPrice']

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
# Displaying the updated DataFrame
print("\nFrequency and Monetary Calculation")
print(df[['CustomerID', 'Recency', 'Frequency', 'Monetary']].head())

In [ ]:
# Calculating unique values in 'Frequency' to check if there is any anamoly in our column
unique_frequencies = df['Frequency'].unique()

# Displaying the unique values
print("Unique Values in Frequency:")
print(unique_frequencies)


In [ ]:
from scipy.stats import skew
#checking skewness because there is nothing else wrong in this column but it is giving us some wrong graphs
# Calculating skewness of 'Frequency'
frequency_skewness = skew(df['Frequency'])
# Displaying the skewness value
print("Skewness of Frequency:", frequency_skewness)


In [ ]:
#for the removel of skewness we will take logrithm of our column
import numpy as np

# Logarithmic transformation of 'Frequency'
df['Frequency_Log'] = np.log1p(df['Frequency'])

# Calculating skewness of the transformed 'Frequency'
frequency_log_skewness = skew(df['Frequency_Log'])

# Displaying the skewness value after transformation
print("Skewness after Logarithmic Transformation:", frequency_log_skewness)


Now as we have less than 1 skewness now we can proceed

In [ ]:
# Specify the number of bins (e.g., 4)
num_bins = 4
frequency_bins = pd.cut(df['Frequency_Log'], bins=num_bins, labels=['1', '2', '3', '4'])

# Assigning the new bin labels to a column
df['Frequency_Quartile'] = frequency_bins.astype(str)

# Displaying the updated DataFrame with the transformed and binned 'Frequency'
print("\nUpdated DataFrame with Transformed and Binned Frequency")
print(df[['CustomerID', 'Recency', 'Frequency_Log', 'Monetary', 'Frequency_Quartile']].head())


In [ ]:
# Calculating unique values in 'Frequency_Quartile'
unique_frequency_quartiles = df['Frequency_Quartile'].unique()

# Displaying the unique values
print("Unique Values in Frequency_Quartile:")
print(unique_frequency_quartiles)


In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
# Convert 'Frequency_Quartile' to integer type
df['Frequency_Quartile'] = df['Frequency_Quartile'].astype(int)

# Displaying the updated DataFrame with the transformed data types
print("\nUpdated DataFrame with Transformed Data Types")
print(df.dtypes)


In [ ]:
# checking skewness of 'Recency'
recency_skewness = skew(df['Recency'])

# Displaying the skewness value
print("Skewness of Recency:", recency_skewness)


In [ ]:
# Logarithmic transformation of 'Recency'
df['Recency_Log'] = np.log1p(df['Recency'])

# checking skewness after transformation
recency_log_skewness = skew(df['Recency_Log'])
print("Skewness of Recency after Logarithmic Transformation:", recency_log_skewness)

In [ ]:
# Calculating quartiles for the transformed 'Recency'
recency_quartiles = pd.qcut(df['Recency_Log'], q=4, labels=['4', '3', '2', '1'])

# Assigning quartiles to a new column
df['Recency_Quartile'] = recency_quartiles.astype(str)

# Displaying the updated DataFrame with the transformed and segmented 'Recency'
print("\nUpdated DataFrame with Transformed and Segmented Recency")
print(df[['CustomerID', 'Recency_Log', 'Recency_Quartile']].head())


In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
# Convert 'Recency_Quartile' to integer type
df['Recency_Quartile'] = df['Recency_Quartile'].astype(int)

# Displaying the updated DataFrame with the transformed data types
print("\nUpdated DataFrame with Transformed Data Types")
print(df.dtypes)


In [ ]:
df.isnull().sum()

We have alread dealt with skewness we will remove the skewness fromthis column of Monetary too

In [ ]:
# checking skewness of 'Monetary'
Monetary_skewness = skew(df['Monetary'])

# Displaying the skewness value
print("Skewness of Monetary:", recency_skewness)

In [ ]:
# adding a small constant to handle potential zero or negative values
constant = 1
df['Monetary_Log'] = np.log1p(df['Monetary'] + constant)

# checking skewness after transformation
monetary_log_skewness = skew(df['Monetary_Log'])
print("Skewness of Monetary after Logarithmic Transformation:", monetary_log_skewness)

In [ ]:
# adding a constant to handle negative values
constant_monetary = abs(df['Monetary'].min()) + 1
df['Monetary_Log'] = np.log1p(df['Monetary'] + constant_monetary)

# checking skewness after transformation
monetary_log_skewness = skew(df['Monetary_Log'])
print("Skewness of Monetary after Logarithmic Transformation:", monetary_log_skewness)

After cheking skwness we can see abnormality in our data and can guess somthing is wrong with it so will will count if thre anre some negative vakue in our data whic is not being removed with the addition of a constant value

In [ ]:
# Count of negative values in 'Monetary'
negative_values_count = (df['Monetary'] < 0).sum()
print("Count of Negative Values in Monetary:", negative_values_count)


In [ ]:
# Set negative values in 'Monetary' to zero
df['Monetary'] = df['Monetary'].apply(lambda x: max(x, 0))

# Logarithmic transformation of 'Monetary'
df['Monetary_Log'] = np.log1p(df['Monetary'])

# checking skewness after transformation
monetary_log_skewness = skew(df['Monetary_Log'])
print("Skewness of Monetary after Logarithmic Transformation:", monetary_log_skewness)

No we have prepared our Monetary column too

In [ ]:
# Calculating quartiles for the transformed 'Monetary'
monetary_quartiles = pd.qcut(df['Monetary_Log'], q=4, labels=['1', '2', '3', '4'])

# Assigning quartiles to a new column
df['Monetary_Quartile'] = monetary_quartiles.astype(str)

# Displaying the updated DataFrame with the transformed and segmented 'Monetary'
print("\nUpdated DataFrame with Transformed and Segmented Monetary")
print(df[['CustomerID', 'Monetary_Log', 'Monetary_Quartile']].head())


In [ ]:
df.columns

In [ ]:
df.dtypes
df.isnull().sum()

# Now Let's creat **RFM score**

In [ ]:
# Assigning scores for Recency, Frequency, and Monetary
df['Recency_Score'] = pd.to_numeric(df['Recency_Quartile'])
df['Frequency_Score'] = pd.to_numeric(df['Frequency_Quartile'])
df['Monetary_Score'] = pd.to_numeric(df['Monetary_Quartile'])

# Combine scores to create RFM score
df['RFM_Score'] = df['Recency_Score'].astype(str) + df['Frequency_Score'].astype(str) + df['Monetary_Score'].astype(str)

# Displaying the updated DataFrame with RFM scores
print("\nUpdated DataFrame with RFM Scores")
print(df[['CustomerID', 'Recency_Score', 'Frequency_Score', 'Monetary_Score', 'RFM_Score']].head())


In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
# Convert RFM_Score to numeric
df['RFM_Score'] = pd.to_numeric(df['RFM_Score'])

# Displaying the updated DataFrame with numeric RFM scores
print("\nUpdated DataFrame with Numeric RFM Scores")
print(df[['CustomerID', 'Recency_Score', 'Frequency_Score', 'Monetary_Score', 'RFM_Score']].head())


In [ ]:
df.columns

Now we will segment our customer according to there RFM values

In [ ]:
# Adjusting segmentation based on sample RFM scores
segment_dict = {
    'High-Value Customers': ['444'],
    'Potential Loyal Customers': ['344', '434'],
    'Big Spenders': ['424', '423', '414', '412', '411', '414', '434'],
    'Recent Customers': ['244', '234', '224', '213', '214'],
    'Churn Risk Customers': ['111', '113', '121', '122', '131', '132']
}

# Applying segmentation to create a new 'Segment' column
def assign_segment(rfm_score):
    for segment, score_list in segment_dict.items():
        if rfm_score in score_list:
            return segment
    return 'Other'  # Default segment if not found in any defined segments

# Applying segmentation to create a new 'Segment' column
df['Segment'] = df['RFM_Score'].astype(str).apply(assign_segment)

# Displaying the updated DataFrame with the 'Segment' column
print("\nUpdated DataFrame with Segmentation")
print(df[['CustomerID', 'RFM_Score', 'Segment']].head())


In [ ]:
# To checking if all the records are correctly assignted to correct segment let's Displaying two samples from each segment with index
for segment in segment_dict.keys():
    segment_samples = df[df['Segment'] == segment].sample(2).reset_index()
    print(f"\nSamples from {segment} segment:")
    print(segment_samples[['index', 'CustomerID', 'RFM_Score', 'Segment']])



In [ ]:
# Adjusting segmentation based on RFM scores
segment_dict = {
    'High-Value Customers': ['444'],
    'Potential Loyal Customers': ['344', '434','433','334','332','333'],
    'Big Spenders': ['424', '423', '414', '412', '411', '434', '442', '441', '443','432','114','324'],
    'Recent Customers': ['244', '234', '224', '213', '214', '431','134', '112', '223','313', '314', '322', '323', '311', '312', '222', '422', '421', '212', '211', '413', '124', '123', '221', '321', '331',  '233', '231', '232'],
    'Churn Risk Customers': ['111', '113', '121', '122', '131', '132','133']
}

# Function to Assigning segment based on RFM Score
def assign_segment(rfm_score):
    for segment, scores in segment_dict.items():
        if rfm_score in scores:
            return segment
    return 'Other'  # Default for any other case

# Applying segmentation to create a new 'Segment' column
df['Segment'] = df['RFM_Score'].astype(str).apply(assign_segment)

# Displaying the updated DataFrame with the 'Segment' column
print("\nUpdated DataFrame with Segmentation")
print(df[['CustomerID', 'RFM_Score', 'Segment']].head())


In [ ]:
# Displaying total count of each segment
segment_counts = df['Segment'].value_counts().reset_index()
segment_counts.columns = ['Segment', 'Count']
print("\nTotal Count of Each Segment:")
print(segment_counts)


In [ ]:
df.columns

In [ ]:
# Creating a bar plot for the distribution of customers across segments
plt.figure(figsize=(15, 6))
sns.countplot(x='Segment', data=df, palette='viridis')

# Set plot labels and title
plt.xlabel('Customer Segment')
plt.ylabel('Number of Customers')
plt.title('Distribution of Customers Across Segments')

# showing the total count on each bar
for index, value in enumerate(df['Segment'].value_counts()):
    plt.text(index, value + 5, str(value), ha='center', va='bottom')

# showing the plot
plt.show()








analyze key metrics such as average Recency, Frequency, and Monetary values within each segment. let's Calculating and compare the means for each metric.

In [ ]:
# Group by 'Simplified_Segment_Name' and Calculating the mean for each metric
segment_metrics = df.groupby('Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean'
}).reset_index()

# Displaying the Calculatingd means
print("\nAverage Metrics Within Each Segment:")
print(segment_metrics)


In [ ]:
#there is i think out liers in churn risk why there is 230% is being shown there to chek that we need to chek for any outliers
# Filter data for Churn Risk Customers segment
churn_risk_data = df[df['Segment'] == 'Churn Risk Customers']

# Set the style for the plot
sns.set(style="whitegrid")

# Creating a box plot for Recency in Churn Risk Customers segment
plt.figure(figsize=(10, 6))
sns.boxplot(x=churn_risk_data['Recency'], palette='viridis')

# Set plot labels and title
plt.xlabel('Recency in Churn Risk Customers')
plt.title('Distribution of Recency in Churn Risk Customers Segment')

# Show the plot
plt.show()


visualize these average metrics within each segment to gain a clearer understanding.Let's create bar plots for each metric to compare the values across segments

In [ ]:
# Calculating average recency for each segment
avg_recency = df.groupby('Segment')['Recency'].mean().reset_index()

# Set the style for the plot
sns.set(style="whitegrid")

# Creatinga bar plot for average Recency across segments
plt.figure(figsize=(15, 6))
ax = sns.barplot(x='Segment', y='Recency', data=avg_recency, palette='viridis')

# Annotate the bars with their respective average recency values
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}%', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points', fontsize=10, color='black')

# Set plot labels and title
plt.xlabel('Customer Segment')
plt.ylabel('Average Recency')
plt.title('Average Recency Across Customer Segments')

# showing the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Define custom colors for each segment
segment_colors = {
    'High-Value Customers': 'skyblue',
    'Potential Loyal Customers': 'lightcoral',
    'Big Spenders': 'lightgreen',
    'Recent Customers': 'lightsalmon',
    'Churn Risk Customers': 'lightblue'
}

# Creating a pie chart for Recency across segments
plt.figure(figsize=(15, 15))

# Data for Recency
recency_data = df.groupby('Segment')['Recency'].mean()
labels = recency_data.index
sizes = recency_data.values

# Creating a pie chart with custom colors
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=[segment_colors[segment] for segment in labels])

# Draw a circle at the center of the pie to make it look like a donut
centre_circle = plt.Circle((0, 0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

# Set plot title
plt.title('Average Recency Across Customer Segments')

# showing the plot
plt.show()


Average Frequency graph

In [ ]:
# Set the style for the plot
sns.set(style="whitegrid")

# Bar plot for Average Frequency
plt.figure(figsize=(15, 6))
sns.barplot(x='Segment', y='Frequency', data=segment_metrics, palette='viridis')
plt.title('Average Frequency Within Each Segment')
plt.xlabel('Customer Segment')
plt.ylabel('Average Frequency')
plt.show()


Average Monetary Value Within Each Segment

In [ ]:

# Set the style for the plot
sns.set(style="whitegrid")

# Bar plot for Average Monetary Value
plt.figure(figsize=(15, 6))
sns.barplot(x='Segment', y='Monetary', data=segment_metrics, palette='viridis')
plt.title('Average Monetary Value Within Each Segment')
plt.xlabel('Simplified Customer Segment')
plt.ylabel('Average Monetary Value')
plt.show()


High-Value customers may not be frequent shoppers but have moderate spending.
Potential Loyal customers show recent activity, higher frequency, and moderate spending.
Big Spenders are very recent and have both high frequency and high spending.

**Targeted marketing strategies**:

High-Value customers might be incentivized to increase their frequency, while efforts for Potential Loyal customers Should focus on retaining their loyalty. Big Spenders could be targeted with exclusive offers to maintain their high spending levels.

# **Demographics**

In [ ]:
# Group data by Country and Simplified_Segment_Name, and Calculating customer counts
segment_distribution = df.groupby(['Country', 'Segment']).size().reset_index(name='CustomerCount')

# Displaying the first few rows of the resulting DataFrame
print(segment_distribution.head())


In [ ]:
# Set the style for the plots
sns.set(style="whitegrid")

# Creatinga bar plot to visualize the distribution of customer segments by country
plt.figure(figsize=(12, 8))
sns.barplot(x='Country', y='CustomerCount', hue='Segment', data=segment_distribution)
plt.title('Distribution of Customer Segments by Country')
plt.xlabel('Country')
plt.ylabel('Customer Count')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.legend(title='Customer Segment')
plt.show()


In [ ]:
import numpy as np

# Set the style for the plots
sns.set(style="whitegrid")

# Creating a bar plot with a logarithmic scale for the y-axis
plt.figure(figsize=(12, 8))
sns.barplot(x='Country', y='CustomerCount', hue='Segment', data=segment_distribution)
plt.title('Distribution of Customer Segments by Country')
plt.xlabel('Country')
plt.ylabel('Customer Count (log scale)')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.yscale('log')  # Set y-axis to logarithmic scale
plt.legend(title='Customer Segment')
plt.show()


Using heat map for better understanding

In [ ]:
# Pivot the data to create a matrix for the heatmap
heatmap_data = segment_distribution.pivot(index='Country', columns='Segment', values='CustomerCount')

# Set the style for the plot
sns.set(style="whitegrid")

# Creatinga heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(heatmap_data, annot=True, fmt=".2f", cmap="YlGnBu", linewidths=.5)
plt.title('Distribution of Customer Segments by Country (Heatmap)')
plt.xlabel('Customer Segment')
plt.ylabel('Country')
plt.show()


In [ ]:
import plotly.express as px

# Creating a choropleth map
fig = px.choropleth(
    segment_distribution,
    locations='Country',
    locationmode='country names',
    color='Segment',
    title='Customer Segment Distribution by Country',
    color_discrete_map={'High-Value': 'blue', 'Potential Loyal': 'green', 'Big Spenders': 'red'},
    labels={'Segment': 'Customer Segment'}
)

# showing the map
fig.show()


In [ ]:
# Group data by Country and Simplified_Segment_Name, and Calculating customer counts
segment_distribution = df.groupby(['Country', 'Segment']).size().reset_index(name='CustomerCount')

# Displaying the resulting DataFrame
print(segment_distribution)


In [ ]:
# Set the option to Displaying all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Displaying the resulting DataFrame
print(segment_distribution)

# Reset the options to the default values if needed
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')


In [ ]:
df.columns

# **Suggestions:**
1. **United Kingdom Insights:**
   - Given the substantial customer count in the "High-Value" segment in the United Kingdom, consider implementing personalized promotions or loyalty programs to further engage and retain this large customer base locally.

2. **Potential Loyal Customers:**
   - Identify common characteristics or behaviors among customers in the "Potential Loyal" segment across various countries.
   - Target these customers with promotions, such as exclusive discounts, to encourage repeat purchases and foster loyalty.

3. **Big Spenders Worldwide:**
   - Recognize the global presence of the "Big Spenders" segment and tailor international marketing strategies to attract and retain high-value customers globally.

4. **Understanding Recent Customers:**
   - Investigate factors influencing the lower customer count in the "Recent Customers" segment across countries, such as seasonality or external events.
   - Implement marketing or outreach campaigns to re-engage recent customers and encourage additional purchases.

5. **Diversification in Other Countries:**
   - Explore business expansion opportunities in countries where certain segments are underrepresented.
   - Conduct market research to understand the preferences and behaviors of customers in these countries and tailor marketing strategies accordingly.





In [ ]:
# Group by InvoiceNo and aggregate products
invoice_products = df.groupby('CustomerID')['Description'].unique().reset_index()

# Displaying the resulting DataFrame
print(invoice_products)


In [ ]:
df.columns


In [ ]:
# Group by InvoiceNo and aggregate the Description into lists
df['Purchased_Products'] = df.groupby('InvoiceNo')['Description'].agg(list)

# Displaying the first few rows of the DataFrame with the new 'Purchased_Products' column
print(df[['InvoiceNo', 'Purchased_Products']].tail())


In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
# Print the actual values in the 'Purchased_Products' column for each segment, handling NaN values
for segment_name, segment_data in df.groupby('Segment'):
    print(f"\n--- Purchased Products for {segment_name} ---")
    print(segment_data['Purchased_Products'].dropna())


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style for the plot
sns.set(style="whitegrid")

# Analyze the most popular products within each customer segment
for segment_name, segment_data in df.groupby('Segment'):
    # Drop NaN values from the 'Purchased_Products' column
    products = segment_data['Purchased_Products'].dropna()

    if not products.empty:
        # Concatenate the lists of purchased products for each customer in the segment
        all_products = [product for products in products for product in products]
        # Count the occurrences of each product
        product_counts = pd.Series(all_products).value_counts().head(5)  # Displaying the top 5 products

        # Creatinga horizontal bar plot
        plt.figure(figsize=(10, 6))
        sns.barplot(x=product_counts, y=product_counts.index, palette='viridis')
        # Set plot labels and title
        plt.xlabel('Number of Purchases')
        plt.ylabel('Product')
        plt.title(f'Top 5 Products for {segment_name}')
        # Show the plot
        plt.show()
    else:
        print(f"No purchased products for {segment_name}.")


# **Result:**

- **High-Value Customers:** They seem to frequently purchase items like "DOTCOM POSTAGE," "JUMBO BAG RED RETROSPOT," and "WHITE HANGING HEART T-LIGHT HOLDER."

- **Potential Loyal Customers:** Popular products include "WHITE HANGING HEART T-LIGHT HOLDER," "REGENCY CAKESTAND 3 TIER," and "LUNCH BAG RED RETROSPOT."

- **Big Spenders:** The top products for this segment include "PACK OF 72 RETROSPOT CAKE CASES," "JUMBO BAG RED RETROSPOT," and "REGENCY CAKESTAND 3 TIER."

- **Recent Customers:** There are no specific popular products identified for this segment based on the current data.



**Are there specific products or categories that the "Big Spenders" segment tends to purchase more frequently?**

In [ ]:
# Filter the DataFrame for the "Big Spenders" segment
big_spenders_df = df[df['Segment'] == 'Big Spenders']

# Explore the most frequently purchased products
top_products = big_spenders_df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)

# Explore the most frequently purchased categories (we have 'StockCode' represents categories)
top_categories = big_spenders_df.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False).head(10)

# Displaying the top products and categories
print("\nTop Products Purchased by Big Spenders:")
print(top_products)

print("\nTop Categories Purchased by Big Spenders:")
print(top_categories)


In [ ]:
# Set up the plotting environment
plt.figure(figsize=(12, 6))

# Plotting the top products
plt.subplot(1, 2, 1)
top_products.plot(kind='bar', color='skyblue')
plt.title('Top Products Purchased by Big Spenders')
plt.xlabel('Product Description')
plt.ylabel('Total Quantity Purchased')

# Plotting the top categories
plt.subplot(1, 2, 2)
top_categories.plot(kind='bar', color='lightcoral')
plt.title('Top Categories Purchased by Big Spenders')
plt.xlabel('Stock Code')
plt.ylabel('Total Quantity Purchased')

# Adjusting layout
plt.tight_layout()

# showing the plots
plt.show()






In [ ]:
# Set up the plotting environment
plt.figure(figsize=(12, 6))


top_products.plot(kind='bar', color='skyblue')
plt.title('Top Products Purchased by Big Spenders')
plt.xlabel('Product Description')
plt.ylabel('Total Quantity Purchased')



# showing the plots
plt.show()






**specific products or categories that the "Big Spenders" segment tends to purchase more frequently?**

In [ ]:
# Filter data for Potential Loyal Customers
potential_loyal_data = df[df['Segment'] == 'Potential Loyal Customers']

# Calculating the quantity of each product
potential_loyal_products = potential_loyal_data.groupby('Description')['Quantity'].sum().sort_values(ascending=False)

# Displaying the top products
print("Top Products Purchased by Potential Loyal Customers:")
print(potential_loyal_products.head(10))

# Calculating the quantity of each category (StockCode)
potential_loyal_categories = potential_loyal_data.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False)

# Displaying the top categories
print("\nTop Categories Purchased by Potential Loyal Customers:")
print(potential_loyal_categories.head(10))


In [ ]:
# Set up subplots
plt.figure(figsize=(12, 6))

# Plotting the top products
plt.subplot(1, 2, 1)
potential_loyal_products.head(10).plot(kind='bar', color='skyblue')
plt.title('Top Products Purchased by Potential Loyal Customers')
plt.xlabel('Product Description')
plt.ylabel('Quantity')
plt.xticks(rotation=45, ha='right')

# Plotting the top categories
plt.subplot(1, 2, 2)
potential_loyal_categories.head(10).plot(kind='bar', color='lightcoral')
plt.title('Top Categories Purchased by Potential Loyal Customers')
plt.xlabel('StockCode')
plt.ylabel('Quantity')
plt.xticks(rotation=45, ha='right')

# Adjusting the layout
plt.tight_layout()

# showing the plots


specific products or categories that the "High-Value Customers" segment tends to purchase more frequently?

In [ ]:
# Filter data for High-Value Customers
high_value_data = df[df['Segment'] == 'High-Value Customers']

# Calculating the quantity of each product
high_value_products = high_value_data.groupby('Description')['Quantity'].sum().sort_values(ascending=False)

# Displaying the top products
print("\nTop Products Purchased by High-Value Customers:")
print(high_value_products.head(10))

# Calculating the quantity of each category (StockCode)
high_value_categories = high_value_data.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False)

# Displaying the top categories
print("\nTop Categories Purchased by High-Value Customers:")
print(high_value_categories.head(10))


In [ ]:
# Set up subplots
plt.figure(figsize=(12, 6))

# Plotting the top products
plt.subplot(1, 2, 1)
high_value_products.head(10).plot(kind='bar', color='skyblue')
plt.title('Top Products Purchased by High-Value Customers')
plt.xlabel('Product Description')
plt.ylabel('Quantity')
plt.xticks(rotation=45, ha='right')

# Plotting the top categories
plt.subplot(1, 2, 2)
high_value_categories.head(10).plot(kind='bar', color='lightcoral')
plt.title('Top Categories Purchased by High-Value Customers')
plt.xlabel('StockCode')
plt.ylabel('Quantity')
plt.xticks(rotation=45, ha='right')

# Adjusting the layout
plt.tight_layout()

# showing the plots
plt.show()


**Loyalty and Retention:**
**What is the average time gap between purchases for the "Big Spenders" segment?**


In [ ]:
# we have df is we have DataFrame with the relevant columns
big_spenders_data = df[df['Segment'] == 'Big Spenders']

# Sort the data by CustomerID and InvoiceDate
big_spenders_data = big_spenders_data.sort_values(by=['CustomerID', 'InvoiceDate'])

# Calculating the time gap between consecutive purchases for each customer
big_spenders_data['Time_Gap'] = big_spenders_data.groupby('CustomerID')['InvoiceDate'].diff()

# Calculating the average time gap for the "Big Spenders" segment
average_time_gap = big_spenders_data['Time_Gap'].mean()

# Displaying the result
print("Average Time Gap between Purchases for Big Spenders:", average_time_gap)


In [ ]:
# Set the threshold for the extended period (e.g., 3 months)
extended_period_threshold = 3

# Identify high-value customers
high_value_customers = df[df['Segment'] == 'High-Value Customers']['CustomerID'].unique()

# Filter data for high-value customers
high_value_data = df[df['CustomerID'].isin(high_value_customers)]

# Calculating the purchase frequency for each high-value customer within the extended period
consistent_purchase_counts = high_value_data.groupby('CustomerID')['InvoiceDate'].diff().dt.days.le(extended_period_threshold).sum()

# Displaying the result
print("Number of High-Value Customers with Consistent Purchases:", consistent_purchase_counts)


In [ ]:
# Set the threshold for the extended period (e.g., 1 months)
extended_period_threshold = 1

# Creating a DataFrame to store the results
consistent_purchase_results = pd.DataFrame(columns=['CustomerID', 'Segment', 'ConsistentPurchaseCount'])

# Iterate over each segment
for segment in segment_dict.keys():
    # Identify customers in the current segment
    segment_customers = df[df['Segment'] == segment]['CustomerID'].unique()

    # Filter data for the current segment
    segment_data = df[df['CustomerID'].isin(segment_customers)]

    # Calculating the purchase frequency for each customer within the extended period
    consistent_purchase_count = segment_data.groupby('CustomerID')['InvoiceDate'].diff().dt.days.le(extended_period_threshold).sum()

    # Append the result to the DataFrame
    consistent_purchase_results = consistent_purchase_results.append({'CustomerID': segment_customers[0], 'Segment': segment, 'ConsistentPurchaseCount': consistent_purchase_count}, ignore_index=True)

# Displaying the results
print("Consistent Purchase Count for Each Segment:")
print(consistent_purchase_results)


**Recomendations on Frequency**

**Churn Risk Customers:**

In [ ]:
# Calculating average frequency for Churn Risk Customers
churn_avg_frequency = df[df['Segment'] == 'Churn Risk Customers']['Frequency'].mean()
print("Average Frequency for Churn Risk Customers:", churn_avg_frequency)


**Recent Customers:**


In [ ]:
# Calculating average frequency for Recent Customers
recent_avg_frequency = df[df['Segment'] == 'Recent Customers']['Frequency'].mean()
print("Average Frequency for Recent Customers:", recent_avg_frequency)


**Potential Loyal Customers:**

In [ ]:
# Calculating average frequency for Potential Loyal Customers
loyal_avg_frequency = df[df['Segment'] == 'Potential Loyal Customers']['Frequency'].mean()
print("Average Frequency for Potential Loyal Customers:", loyal_avg_frequency)


In [ ]:
# Calculating average frequency for each segment
churn_avg_frequency = df[df['Segment'] == 'Churn Risk Customers']['Frequency'].mean()
recent_avg_frequency = df[df['Segment'] == 'Recent Customers']['Frequency'].mean()
loyal_avg_frequency = df[df['Segment'] == 'Potential Loyal Customers']['Frequency'].mean()

# Plotting the average frequencies with annotations
plt.figure(figsize=(10, 6))
bars = plt.bar(['Churn Risk', 'Recent Customers', 'Potential Loyal'], [churn_avg_frequency, recent_avg_frequency, loyal_avg_frequency], color=['lightcoral', 'skyblue', 'gold'])

# addinging annotations on each bar
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom', fontsize=10)

plt.title('Average Frequency for Different Customer Segments')
plt.xlabel('Customer Segments')
plt.ylabel('Average Frequency')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Calculating average frequency for each segment
churn_avg_frequency = df[df['Segment'] == 'Churn Risk Customers']['Frequency'].mean()
recent_avg_frequency = df[df['Segment'] == 'Recent Customers']['Frequency'].mean()
loyal_avg_frequency = df[df['Segment'] == 'Potential Loyal Customers']['Frequency'].mean()
high_value_avg_frequency = df[df['Segment'] == 'High-Value Customers']['Frequency'].mean()
big_spenders_avg_frequency = df[df['Segment'] == 'Big Spenders']['Frequency'].mean()

# Plotting the average frequencies with annotations
plt.figure(figsize=(12, 8))
bars = plt.bar(['Churn Risk', 'Recent Customers', 'Potential Loyal', 'High-Value Customers', 'Big Spenders'],
               [churn_avg_frequency, recent_avg_frequency, loyal_avg_frequency, high_value_avg_frequency, big_spenders_avg_frequency],
               color=['lightcoral', 'skyblue', 'gold', 'mediumseagreen', 'orchid'])

# Adding annotations on each bar
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom', fontsize=10)

plt.title('Average Frequency for Different Customer Segments')
plt.xlabel('Customer Segments')
plt.ylabel('Average Frequency')
plt.show()


**Location wise Frequency**

In [ ]:
# Group data by Country and Calculating average frequency
country_frequency = df.groupby('Country')['Frequency'].mean().reset_index()

# Sort by average frequency in descending order
sorted_countries = country_frequency.sort_values(by='Frequency', ascending=False)

# Extract top 5 and least 5 countries
top_5_countries = sorted_countries.head(5)
least_5_countries = sorted_countries.tail(5)

# Displaying the results
print("Top 5 Countries (Highest Average Frequency):")
print(top_5_countries)

print("\nLeast 5 Countries (Lowest Average Frequency):")
print(least_5_countries)


In [ ]:
import matplotlib.pyplot as plt

# Set up the plotting environment
plt.figure(figsize=(12, 6))

# Plot the top 5 countries with data values
plt.subplot(1, 2, 1)
bars = plt.bar(top_5_countries['Country'], top_5_countries['Frequency'], color='skyblue')
plt.title('Top 5 Countries (Highest Average Frequency)')
plt.xlabel('Country')
plt.ylabel('Average Frequency')

# Displaying data values on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

# Plot the least 5 countries with data values
plt.subplot(1, 2, 2)
bars = plt.bar(least_5_countries['Country'], least_5_countries['Frequency'], color='lightcoral')
plt.title('Least 5 Countries (Lowest Average Frequency)')
plt.xlabel('Country')
plt.ylabel('Average Frequency')

# Displaying data values on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

# Adjusting the layout
plt.tight_layout()

# Show the plots
plt.show()


**Monetary:**

In [ ]:
# Calculating average monetary value for each segment
average_monetary = df.groupby('Segment')['Monetary'].mean().reset_index()

# Set up the plotting environment
plt.figure(figsize=(12, 6))

# Plotting Average Monetary Value for Each Segment
bar_plot = sns.barplot(x='Segment', y='Monetary', data=average_monetary, palette='viridis')
plt.title('Average Monetary Value for Each Segment')
plt.xlabel('Segment')
plt.ylabel('Average Monetary Value')
plt.xticks(rotation=45, ha='right')

# Annotate with numbers on top of the bars
for index, value in enumerate(average_monetary['Monetary']):
    bar_plot.text(index, value, round(value, 2), ha='center', va='bottom', fontsize=10)

# showing the plot
plt.show()


**Segments country wise**

In [ ]:
# Find the country with the most customers in each segment
recent_customers_country = df[df['Segment'] == 'Recent Customers']['Country'].value_counts().idxmax()
big_spenders_country = df[df['Segment'] == 'Big Spenders']['Country'].value_counts().idxmax()
churn_risk_country = df[df['Segment'] == 'Churn Risk Customers']['Country'].value_counts().idxmax()
potential_loyal_country = df[df['Segment'] == 'Potential Loyal Customers']['Country'].value_counts().idxmax()
high_value_country = df[df['Segment'] == 'High-Value Customers']['Country'].value_counts().idxmax()

recent_customers_count = df[df['Segment'] == 'Recent Customers']['Country'].value_counts().max()
big_spenders_count = df[df['Segment'] == 'Big Spenders']['Country'].value_counts().max()
churn_risk_count = df[df['Segment'] == 'Churn Risk Customers']['Country'].value_counts().max()
potential_loyal_count = df[df['Segment'] == 'Potential Loyal Customers']['Country'].value_counts().max()
high_value_count = df[df['Segment'] == 'High-Value Customers']['Country'].value_counts().max()

print(f"Recent Customers: Country - {recent_customers_country}, Count - {recent_customers_count}")
print(f"Big Spenders: Country - {big_spenders_country}, Count - {big_spenders_count}")
print(f"Churn Risk Customers: Country - {churn_risk_country}, Count - {churn_risk_count}")
print(f"Potential Loyal Customers: Country - {potential_loyal_country}, Count - {potential_loyal_count}")
print(f"High-Value Customers: Country - {high_value_country}, Count - {high_value_count}")


In [ ]:
# Set up the plotting environment
plt.figure(figsize=(15, 8))

# Plotting Recent Customers
plt.subplot(2, 3, 1)
sns.countplot(x='Country', data=df[df['Segment'] == 'Recent Customers'], order=df[df['Segment'] == 'Recent Customers']['Country'].value_counts().index[:5])
plt.title('Recent Customers')

# Plotting Big Spenders
plt.subplot(2, 3, 2)
sns.countplot(x='Country', data=df[df['Segment'] == 'Big Spenders'], order=df[df['Segment'] == 'Big Spenders']['Country'].value_counts().index[:5])
plt.title('Big Spenders')

# Plotting Churn Risk Customers
plt.subplot(2, 3, 3)
sns.countplot(x='Country', data=df[df['Segment'] == 'Churn Risk Customers'], order=df[df['Segment'] == 'Churn Risk Customers']['Country'].value_counts().index[:5])
plt.title('Churn Risk Customers')

# Plotting Potential Loyal Customers
plt.subplot(2, 3, 4)
sns.countplot(x='Country', data=df[df['Segment'] == 'Potential Loyal Customers'], order=df[df['Segment'] == 'Potential Loyal Customers']['Country'].value_counts().index[:5])
plt.title('Potential Loyal Customers')

# Plotting High-Value Customers
plt.subplot(2, 3, 5)
sns.countplot(x='Country', data=df[df['Segment'] == 'High-Value Customers'], order=df[df['Segment'] == 'High-Value Customers']['Country'].value_counts().index[:5])
plt.title('High-Value Customers')

# Adjusting the layout
plt.tight_layout()

# showing the plots
plt.show()


In [ ]:
# Set up the plotting environment
plt.figure(figsize=(15, 12))

# Plotting Recent Customers
plt.subplot(3, 2, 1)
recent_customers_top5 = df[df['Segment'] == 'Recent Customers']['Country'].value_counts().head(5)
sns.barplot(x=recent_customers_top5.index, y=recent_customers_top5.values)
plt.title('Recent Customers - Top 5 Countries')
plt.xlabel('Country')
plt.ylabel('Count')
for i, value in enumerate(recent_customers_top5.values):
    plt.text(i, value, str(value), ha='center', va='bottom')

plt.subplot(3, 2, 2)
recent_customers_least5 = df[df['Segment'] == 'Recent Customers']['Country'].value_counts().tail(5)
sns.barplot(x=recent_customers_least5.index, y=recent_customers_least5.values)
plt.title('Recent Customers - Least 5 Countries')
plt.xlabel('Country')
plt.ylabel('Count')
for i, value in enumerate(recent_customers_least5.values):
    plt.text(i, value, str(value), ha='center', va='bottom')


# Plotting Big Spenders
plt.subplot(3, 2, 3)
big_spenders_top5 = df[df['Segment'] == 'Big Spenders']['Country'].value_counts().head(5)
sns.barplot(x=big_spenders_top5.index, y=big_spenders_top5.values)
plt.title('Big Spenders - Top 5 Countries')
plt.xlabel('Country')
plt.ylabel('Count')
for i, value in enumerate(big_spenders_top5.values):
    plt.text(i, value, str(value), ha='center', va='bottom')

plt.subplot(3, 2, 4)
big_spenders_least5 = df[df['Segment'] == 'Big Spenders']['Country'].value_counts().tail(5)
sns.barplot(x=big_spenders_least5.index, y=big_spenders_least5.values)
plt.title('Big Spenders - Least 5 Countries')
plt.xlabel('Country')
plt.ylabel('Count')
for i, value in enumerate(big_spenders_least5.values):
    plt.text(i, value, str(value), ha='center', va='bottom')

# Plotting Churn Risk Customers
plt.subplot(3, 2, 5)
churn_risk_top5 = df[df['Segment'] == 'Churn Risk Customers']['Country'].value_counts().head(5)
sns.barplot(x=churn_risk_top5.index, y=churn_risk_top5.values)
plt.title('Churn Risk Customers - Top 5 Countries')
plt.xlabel('Country')
plt.ylabel('Count')
for i, value in enumerate(churn_risk_top5.values):
    plt.text(i, value, str(value), ha='center', va='bottom')

plt.subplot(3, 2, 6)
churn_risk_least5 = df[df['Segment'] == 'Churn Risk Customers']['Country'].value_counts().tail(5)
sns.barplot(x=churn_risk_least5.index, y=churn_risk_least5.values)
plt.title('Churn Risk Customers - Least 5 Countries')
plt.xlabel('Country')
plt.ylabel('Count')
for i, value in enumerate(churn_risk_least5.values):
    plt.text(i, value, str(value), ha='center', va='bottom')

# Adjusting the layout
plt.tight_layout()

# showing the plots
plt.show()


In [ ]:
# Set up the plotting environment
plt.figure(figsize=(15, 12))

# Plotting Recent Customers
plt.subplot(3, 2, 1)
recent_customers_top5 = df[(df['Segment'] == 'Recent Customers') & (df['Country'] != 'United Kingdom')]['Country'].value_counts().head(5)
sns.barplot(x=recent_customers_top5.index, y=recent_customers_top5.values)
plt.title('Recent Customers - Top 5 Countries (excluding United Kingdom)')
plt.xlabel('Country')
plt.ylabel('Count')
for i, value in enumerate(recent_customers_top5.values):
    plt.text(i, value, str(value), ha='center', va='bottom')

plt.subplot(3, 2, 2)
recent_customers_least5 = df[df['Segment'] == 'Recent Customers']['Country'].value_counts().tail(5)
sns.barplot(x=recent_customers_least5.index, y=recent_customers_least5.values)
plt.title('Recent Customers - Least 5 Countries')
plt.xlabel('Country')
plt.ylabel('Count')
for i, value in enumerate(recent_customers_least5.values):
    plt.text(i, value, str(value), ha='center', va='bottom')

# Plotting Big Spenders
plt.subplot(3, 2, 3)
big_spenders_top5 = df[(df['Segment'] == 'Big Spenders') & (df['Country'] != 'United Kingdom')]['Country'].value_counts().head(5)
sns.barplot(x=big_spenders_top5.index, y=big_spenders_top5.values)
plt.title('Big Spenders - Top 5 Countries (excluding United Kingdom)')
plt.xlabel('Country')
plt.ylabel('Count')
for i, value in enumerate(big_spenders_top5.values):
    plt.text(i, value, str(value), ha='center', va='bottom')

plt.subplot(3, 2, 4)
big_spenders_least5 = df[df['Segment'] == 'Big Spenders']['Country'].value_counts().tail(5)
sns.barplot(x=big_spenders_least5.index, y=big_spenders_least5.values)
plt.title('Big Spenders - Least 5 Countries')
plt.xlabel('Country')
plt.ylabel('Count')
for i, value in enumerate(big_spenders_least5.values):
    plt.text(i, value, str(value), ha='center', va='bottom')

# Plotting Churn Risk Customers
plt.subplot(3, 2, 5)
churn_risk_top5 = df[(df['Segment'] == 'Churn Risk Customers') & (df['Country'] != 'United Kingdom')]['Country'].value_counts().head(5)
sns.barplot(x=churn_risk_top5.index, y=churn_risk_top5.values)
plt.title('Churn Risk Customers - Top 5 Countries (excluding United Kingdom)')
plt.xlabel('Country')
plt.ylabel('Count')
for i, value in enumerate(churn_risk_top5.values):
    plt.text(i, value, str(value), ha='center', va='bottom')

plt.subplot(3, 2, 6)
churn_risk_least5 = df[df['Segment'] == 'Churn Risk Customers']['Country'].value_counts().tail(5)
sns.barplot(x=churn_risk_least5.index, y=churn_risk_least5.values)
plt.title('Churn Risk Customers - Least 5 Countries')
plt.xlabel('Country')
plt.ylabel('Count')
for i, value in enumerate(churn_risk_least5.values):
    plt.text(i, value, str(value), ha='center', va='bottom')

# Adjusting the layout
plt.tight_layout()

# showing the plots
plt.show()


**Recency:**

In [ ]:
# we haveyou have a DataFrame named 'df' with a 'Recency' column and a 'Segment' column

# Group by 'Segment' and Calculating the average recency
average_recency = df.groupby('Segment')['Recency'].mean().reset_index()

# Plot the average recency for each segment with numbers on top of the bars
plt.figure(figsize=(10, 6))
bars = plt.bar(average_recency['Segment'], average_recency['Recency'], color='skyblue')
plt.title('Average Recency for Each Segment')
plt.xlabel('Segment')
plt.ylabel('Average Recency')
plt.xticks(rotation=45, ha='right')

# Displaying the numerical values on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom', color='black')

plt.show()


In [ ]:
from prettytable import PrettyTable

# we have you have a DataFrame named 'average_recency' with 'Segment' and 'Recency' columns
table = PrettyTable()
table.field_names = ["Segment", "Average Recency"]

for index, row in average_recency.iterrows():
    table.add_row([row['Segment'], round(row['Recency'], 2)])

print(table)


**Price Relation**

In [ ]:
# we have DataFrame with columns like 'CustomerID', 'UnitPrice', and 'Segment'
# Replace 'Segment' with the actual column name where you have assigned customer segments

# Calculating average price per customer
avg_price_per_customer = df.groupby('CustomerID')['UnitPrice'].mean().reset_index()

# Merge with the original DataFrame to get the 'Segment' information
df_with_segments = pd.merge(avg_price_per_customer, df[['CustomerID', 'Segment']], on='CustomerID', how='left')

# Plot a bar chart for average prices by customer segment
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='Segment', y='UnitPrice', data=df_with_segments, ci=None, palette='viridis')

# adding values on top of the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='center', xytext=(0, 10), textcoords='offset points', fontsize=8, color='black')

plt.title('Average Unit Price by Customer Segment')
plt.xlabel('Customer Segment')
plt.ylabel('Average Unit Price')
plt.show()


In [ ]:
# we have 'df' is we have DataFrame with columns like 'CustomerID', 'UnitPrice', and 'Segment'
# Replace 'Segment' with the actual column name where you have assigned customer segments

# Calculating average price per customer
avg_price_per_customer = df.groupby('CustomerID')['UnitPrice'].mean().reset_index()

# Merge with the original DataFrame to get the 'Segment' information
df_with_segments = pd.merge(avg_price_per_customer, df[['CustomerID', 'Segment']], on='CustomerID', how='left')

# Creatinga subplot with 1 row and 2 columns
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot the average prices by customer segment with switched axes
ax1 = sns.barplot(x='UnitPrice', y='Segment', data=df_with_segments, ci=None, palette='viridis', ax=axes[0])
for p in ax1.patches:
    ax1.annotate(f'{p.get_width():.2f}', (p.get_width(), p.get_y() + p.get_height() / 2.), ha='left', va='center', xytext=(5, 0), textcoords='offset points', fontsize=8, color='black')
ax1.set_title('Average Unit Price by Customer Segment')
ax1.set_xlabel('Average Unit Price')
ax1.set_ylabel('Customer Segment')

# Plot the percentage of customers in each segment
segment_percentage = df['Segment'].value_counts(normalize=True) * 100
ax2 = segment_percentage.plot(kind='pie', autopct='%1.1f%%', colors=sns.color_palette('viridis'), ax=axes[1])
ax2.set_title('Percentage of Customers in Each Segment')

# Adjusting the layout
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
from prettytable import PrettyTable

# Calculating average price per customer
avg_price_per_customer = df.groupby('CustomerID')['UnitPrice'].mean().reset_index()

# Merge with the original DataFrame to get the 'Segment' information
df_with_segments = pd.merge(avg_price_per_customer, df[['CustomerID', 'Segment']], on='CustomerID', how='left')

# Displaying the results in a table
table = PrettyTable()
table.field_names = ['CustomerID', 'Segment', 'Average Unit Price']

for _, row in df_with_segments.iterrows():
    table.add_row([row['CustomerID'], row['Segment'], f'{row["UnitPrice"]:.2f}'])

print("Average Unit Price by Customer Segment:")
print(table)
